In [11]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from google.colab import files
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import ModelCheckpoint
import matplotlib.pyplot as plt

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [12]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df_drop = application_df.drop(["EIN","NAME"],axis=1)

In [13]:
# Determine the number of unique values in each column.
unique_counts =  application_df_drop.nunique()
unique_counts

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [14]:
# Look at APPLICATION_TYPE value counts for binning
application_types=application_df_drop["APPLICATION_TYPE"].value_counts()
application_types

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [15]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`

application_types_to_replace = list(application_types[application_types<1000].index)

# Replace in dataframe
application_df_drop['APPLICATION_TYPE'] = application_df_drop['APPLICATION_TYPE'].replace(application_types_to_replace, "Other")

# Check to make sure binning was successful
application_df_drop['APPLICATION_TYPE'].value_counts()

T3       27037
Other     2266
T4        1542
T6        1216
T5        1173
T19       1065
Name: APPLICATION_TYPE, dtype: int64

In [16]:
# Look at CLASSIFICATION value counts for binning
classification_types = application_df_drop["CLASSIFICATION"].value_counts()
classification_types

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [17]:
# You may find it helpful to look at CLASSIFICATION value counts >1
classification_types_filtered=classification_types[classification_types>1]
classification_types_filtered

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: CLASSIFICATION, dtype: int64

In [18]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
#  YOUR CODE GOES HERE
classifications_to_replace=list(classification_types[classification_types<1000].index)
# Replace in dataframe

application_df_drop['CLASSIFICATION'] = application_df_drop['CLASSIFICATION'].replace(classifications_to_replace,"Other")

# Check to make sure binning was successful
application_df_drop['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [19]:
# Convert categorical data to numeric with `pd.get_dummies`
application_df_dummies=pd.get_dummies(application_df_drop)
application_df_dummies.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,AFFILIATION_CompanySponsored,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1,108590,1,0,0,1,0,0,0,0,...,1,0,0,0,0,0,0,0,1,0
2,1,5000,0,0,0,0,0,1,0,1,...,0,0,0,0,0,0,0,0,1,0
3,1,6692,1,0,0,1,0,0,0,1,...,0,1,0,0,0,0,0,0,1,0
4,1,142590,1,0,0,1,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0


In [20]:
# Split our preprocessed data into our features and target arrays
Y=application_df_dummies["IS_SUCCESSFUL"].values
X=application_df_dummies.drop("IS_SUCCESSFUL",axis=1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, Y_train, Y_test = train_test_split(X,
                                                   Y,
                                                   random_state= 72)

In [21]:
print("Shape of X (features):", X.shape)
print("Shape of Y (target):",Y.shape)

Shape of X (features): (34299, 40)
Shape of Y (target): (34299,)


In [22]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

Compile, Train and Evaluate the Model

In [34]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features=len(X_train_scaled[1])

print(number_input_features)

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=30,activation="relu",input_dim=40))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=20, activation="relu"))

# third hidden layer
nn.add(tf.keras.layers.Dense(units=10, activation="relu"))

# forth hidden layer
nn.add(tf.keras.layers.Dense(units=15, activation="tanh"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

40
Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_20 (Dense)            (None, 30)                1230      
                                                                 
 dense_21 (Dense)            (None, 20)                620       
                                                                 
 dense_22 (Dense)            (None, 10)                210       
                                                                 
 dense_23 (Dense)            (None, 15)                165       
                                                                 
 dense_24 (Dense)            (None, 1)                 16        
                                                                 
Total params: 2241 (8.75 KB)
Trainable params: 2241 (8.75 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [35]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [36]:
checkpoint_path = "training_checkpoints/cp-{epoch:04d}.ckpt"
checkpoint_callback=ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weight_only=True,
    save_freq="epoch",
    periods=5)

In [37]:
#Train the Model with the Callback:
history = nn.fit(
    X_train_scaled,
    Y_train,
    epochs=100,
    batch_size=32,
    callbacks=[checkpoint_callback])

Epoch 1/100
788/804 [============================>.] - ETA: 0s - loss: 0.5809 - accuracy: 0.7135
Epoch 1: saving model to training_checkpoints/cp-0001.ckpt
804/804 [==============================] - 2s 2ms/step - loss: 0.5806 - accuracy: 0.7140
Epoch 2/100
801/804 [============================>.] - ETA: 0s - loss: 0.5577 - accuracy: 0.7298
Epoch 2: saving model to training_checkpoints/cp-0002.ckpt
804/804 [==============================] - 1s 2ms/step - loss: 0.5578 - accuracy: 0.7298
Epoch 3/100
791/804 [============================>.] - ETA: 0s - loss: 0.5541 - accuracy: 0.7321
Epoch 3: saving model to training_checkpoints/cp-0003.ckpt
804/804 [==============================] - 1s 2ms/step - loss: 0.5547 - accuracy: 0.7314
Epoch 4/100
772/804 [===========================>..] - ETA: 0s - loss: 0.5528 - accuracy: 0.7326
Epoch 4: saving model to training_checkpoints/cp-0004.ckpt
804/804 [==============================] - 2s 2ms/step - loss: 0.5534 - accuracy: 0.7322
Epoch 5/100
781/804 

In [38]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,Y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5632 - accuracy: 0.7252 - 307ms/epoch - 1ms/step
Loss: 0.5632030367851257, Accuracy: 0.7252478003501892


In [39]:
# Export our model to HDF5 file

nn.save("AlphabetSoupCharity_Optimization_1.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
